<a href="https://colab.research.google.com/github/tcglarry/kg_power_line/blob/master/vsb_power_line_first_try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# -*- coding: utf-8 -*-
"""vsb_power_line_eda.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1AZHJmuSwKCP8IOrOg_J96XmoQXDH-O-w
"""

# Step 1
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 110842 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:

# Step 2
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [0]:
src= 'drive/My Drive/kaggle/vsb_power_line/data/'
src_train= 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/train/'

src_valid= 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/validation/'

src_test = 'drive/My Drive/kaggle/vsb_power_line/data/train_test_data/test_data/'

In [0]:
# -*- coding: utf-8 -*-


#==============================================================================
# # # Module
#==============================================================================

from __future__ import print_function


import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import subprocess
import os
import pickle
import sklearn 

from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report


from xgboost.sklearn import XGBClassifier




import keras.backend as K

from keras.applications.inception_v3 import InceptionV3
from keras.applications.densenet import DenseNet121, DenseNet169,DenseNet201, preprocess_input
#from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenetv2 import MobileNetV2
from keras.applications.nasnet import NASNetMobile,NASNetLarge

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,  img_to_array, load_img



from keras.applications.xception import Xception, preprocess_input

from keras.layers import Dense, GlobalAveragePooling2D
from keras.layers import Input, Conv2D, MaxPooling2D, merge, Lambda,UpSampling2D, concatenate, \
Reshape, Dropout,Cropping2D,Activation, BatchNormalization
from keras.models import Model, load_model
import pandas as pd

from keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.callbacks import Callback

from keras.optimizers import Adam

from sklearn.metrics import accuracy_score
from keras import regularizers
#from dummyPy import OneHotEncoder
from keras.utils import to_categorical

from sklearn.metrics import confusion_matrix

import itertools
import time

import gc



In [0]:

train_read_path = src_train ##
val_read_path = src_valid ##

h_size,w_size= 144,288
size, channel = (144,288), 3

num_classes = 2


batch = 32


save_dir =  src
save_file = 'vsb_power_line_DenseNet121_1st_'


In [24]:

#==============================================================================
# # # Train - Generator
#==============================================================================
'''
train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=[0.9, 1.1], 
        fill_mode = 'reflect')
'''

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    width_shift_range=1.0,
    height_shift_range=1.0,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
    directory=train_read_path,
    target_size=(h_size,w_size),
    color_mode="rgb",
    batch_size=batch,
    class_mode="binary",
    shuffle=True,
    seed=42
)


valid_datagen = ImageDataGenerator(
        rescale=1./255)



valid_generator = valid_datagen.flow_from_directory(
    directory=val_read_path,
    target_size=(h_size,w_size),
    color_mode="rgb",
    batch_size=65,
    class_mode="binary",
    shuffle=False,
    seed=42
)


Found 6970 images belonging to 2 classes.
Found 1742 images belonging to 2 classes.


In [0]:


def build_model_2():
    inputs = Input(shape=(h_size,w_size,3))
    #inputs = Lambda(lambda x: x/275. - 1.0)(inputs)
    # create the base pre-trained model
    base_model =DenseNet121(weights='imagenet',input_tensor=inputs,  include_top=True)



    x = base_model.layers[51].output
    print ('shape is = ', x.get_shape())

    x= GlobalAveragePooling2D()(x)
    print ('shape is = ', x.get_shape())
    

    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Dense(64,activation = 'sigmoid')(x)
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    
    x = Dropout(0.2)(x)
    x = Dense(32,activation = 'sigmoid')(x)

    
    '''
    
    x = Dense(128,activation='relu')(x)
    
    x= BatchNormalization()(x)
    x= Activation('relu')(x)

    #x = Dropout(0.3)(x)
    #x = Dense(32,activation='relu')(x)
    
    x = Dense(32,activation='linear', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.01))(x)
    
    
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    '''
    
    x= BatchNormalization()(x)
    x= Activation('relu')(x)
    x = Dropout(0.2)(x)   #BN with dropout ?? try .
    
    output = Dense(1,activation = 'sigmoid')(x)


    model = Model(inputs=base_model.input, outputs=output)
    model.summary()

    return model

In [8]:

#==============================================================================
# # # Train - Fit - ResNet
#==============================================================================

model = build_model_2()

#model = load_model(save_dir+'Stage_1_DenseNet121_binary_generator_1_acc.h5')





33193984/33188688 [==============================] - 1s 0us/step
shape is =  (?, 36, 72, 128)
shape is =  (?, 128)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 144, 288, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 150, 294, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 72, 144, 64)  9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 72, 144, 64)  256         conv1/conv[0][0]            

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
def lr_epoch(epochs):
    learning_rate_ = 0.0102
    decay= 0.01/100.
    lr = learning_rate_ - epochs*decay
    return lr

In [17]:


Step_Train = train_generator.n//train_generator.batch_size

print (train_generator.n, train_generator.batch_size)
Step_Valid = valid_generator.n

#define callback


#define callback
model_save_loss = save_dir+save_file+'_loss_2.h5'
model_save_acc = save_dir+save_file+'_acc_2.h5'
checkpoint_loss = ModelCheckpoint(model_save_loss, monitor='val_loss',save_best_only=True, mode='auto')
checkpoint_acc = ModelCheckpoint(model_save_acc, monitor='val_acc',save_best_only=True, mode='auto')
earlystop = EarlyStopping(monitor='val_loss',  patience=30,  mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=15, min_lr=0.00001,verbose=1)
#（best val accuarcy saved 
lr_scheduler= LearningRateScheduler(lr_epoch,verbose=1)
#callback_list = [checkpoint_loss,checkpoint_acc, earlystop,lr_scheduler]







6970 32


In [12]:
callback_list_1 = [checkpoint_loss,checkpoint_acc, earlystop]


# starts trauning 
his = model.fit_generator(generator=train_generator,
                            steps_per_epoch=Step_Train,
                            validation_data=valid_generator,
                            validation_steps=Step_Valid,
                            epochs=150, verbose = 1,
                             callbacks= callback_list_1)

Epoch 1/150
217/217 [==============================] - 1012s 5s/step - loss: 0.5157 - acc: 0.7506 - val_loss: 0.5454 - val_acc: 0.9402
Epoch 2/150
217/217 [==============================] - 1022s 5s/step - loss: 0.2343 - acc: 0.9394 - val_loss: 0.2420 - val_acc: 0.9393
Epoch 3/150
217/217 [==============================] - 1018s 5s/step - loss: 0.2175 - acc: 0.9391 - val_loss: 0.2472 - val_acc: 0.9402
Epoch 4/150
217/217 [==============================] - 1011s 5s/step - loss: 0.2131 - acc: 0.9399 - val_loss: 0.2293 - val_acc: 0.9393
Epoch 5/150
216/217 [============================>.] - ETA: 0s - loss: 0.2107 - acc: 0.9395

KeyboardInterrupt: ignored

# Start Testing

In [2]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(src_test,target_size=(h_size,w_size),shuffle=False)

filenames = test_generator.filenames
nb_samples = len(filenames)

print (nb_samples)


pred = model.predict_generator(test_generator,nb_samples,verbose=1)


NameError: ignored

In [1]:
pred.shape

NameError: ignored

In [0]:
import pickle 
with open(src+'test_pred.pkl','wb') as handle:
  pickle.dump(pred, handle)



In [0]:
with open(src+'filenames.pkl','wb') as handle:
  pickle.dump(predfilenames, handle)


In [18]:


callback_list_2 = [checkpoint_loss,checkpoint_acc, earlystop,reduce_lr]



# starts trauning 
his_2 = model.fit_generator(generator=train_generator,
                            steps_per_epoch=Step_Train,
                            validation_data=valid_generator,
                            validation_steps=Step_Valid,
                            epochs=100, verbose = 1,
                             callbacks= callback_list_2)


Epoch 1/100
217/217 [==============================] - 1012s 5s/step - loss: 0.2107 - acc: 0.9394 - val_loss: 0.2636 - val_acc: 0.9402
Epoch 2/100
217/217 [==============================] - 1042s 5s/step - loss: 0.2062 - acc: 0.9399 - val_loss: 0.2546 - val_acc: 0.9393
Epoch 3/100
216/217 [============================>.] - ETA: 0s - loss: 0.2008 - acc: 0.9402

KeyboardInterrupt: ignored

In [22]:
#h1 = his.history
h = his_2.history

#h = {}
#for k in h1: h[k] = np.hstack([h1[k], h2[k]])

NameError: ignored

In [19]:
def display_history(history):
    # list all data in history
    print(history.keys())
    # summarize history for accuracy
    plt.plot(history['acc'])
    plt.plot(history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(save_dir+'Stage_1_'+save_file+'_binary_acc.png')
    plt.show()
    # summarize history for loss
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(save_dir+'Stage_1_'+save_file+'_binary_loss.png')
    plt.show()
display_history(h)

NameError: ignored

In [25]:
model.evaluate_generator(valid_generator,verbose=1)

ValueError: ignored

In [0]:
model = load_model(model_save_acc)

In [0]:
model.evaluate_generator(valid_generator,verbose=1)

9/9 [==============================] - 9s 993ms/step


[0.4995784303971699, 0.8761904835700989]